### Checking pipeline for 384 x 384 images

In [3]:
import sys
sys.path.append('/home/jjwhit/rcGAN')

In [2]:
from data.lightning.MassMappingDataModule import MMDataTransform as MMD

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [3]:
import numpy as np
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from typing import Optional
from data.datasets.MM_data import MassMappingDataset_Test, MassMappingDataset_Train, MassMappingDataset_Val
from utils.mri import transforms
from typing import Tuple
import pathlib

In [9]:
kernel =  MMD.compute_fourier_kernel(384)

In [10]:
print(kernel)
kernel.shape

[[ 0.        +0.j         -1.        +0.j         -1.        +0.j
  ... -1.        +0.j         -1.        +0.j
  -1.        +0.j        ]
 [ 1.        +0.j          0.        +1.j         -0.6       +0.8j
  ... -0.8       -0.6j        -0.6       -0.8j
   0.        -1.j        ]
 [ 1.        +0.j          0.6       +0.8j         0.        +1.j
  ... -0.38461538-0.92307692j  0.        -1.j
   0.6       -0.8j       ]
 ...
 [ 1.        +0.j          0.8       -0.6j         0.38461538-0.92307692j
  ...  0.        +1.j          0.38461538+0.92307692j
   0.8       +0.6j       ]
 [ 1.        +0.j          0.6       -0.8j         0.        -1.j
  ... -0.38461538+0.92307692j  0.        +1.j
   0.6       +0.8j       ]
 [ 1.        +0.j          0.        -1.j         -0.6       -0.8j
  ... -0.8       +0.6j        -0.6       +0.8j
   0.        +1.j        ]]


(384, 384)

In [11]:
kappa_test = np.load('/share/gpu0/jjwhit/mass_map_dataset/kappa20_cropped/kappa_val/kappa_run_09556.npy')

In [12]:
kappa_test.shape

(384, 384)

In [14]:
gamma = MMD.forward_model(kappa_test, kernel)

In [18]:
gamma_w_noise =  MMD.noise_maker(5.0, 30, kappa_test)

In [20]:
gamma_w_noise.shape

(384, 384)

In [24]:
pt_kappa  = transforms.to_tensor(kappa_test.astype(np.complex128))

In [25]:
pt_kappa.shape

torch.Size([384, 384, 2])

In [26]:
pt_kappa = pt_kappa.permute(2,0,1)
pt_kappa.shape

torch.Size([2, 384, 384])

In [27]:
pt_gamma = transforms.to_tensor(gamma_w_noise)
pt_gamma = pt_gamma.permute(2, 0, 1)

In [28]:
pt_gamma.shape

torch.Size([2, 384, 384])

In [29]:
normalized_gamma, mean, std = transforms.normalize_instance(pt_gamma)
normalized_gt = transforms.normalize(pt_kappa, mean, std)

In [33]:
std.shape

torch.Size([])

In [1]:
N = 384

In [4]:
import numpy as np
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from typing import Optional
from data.datasets.MM_data import MassMappingDataset_Test, MassMappingDataset_Train, MassMappingDataset_Val
from utils.mri import transforms
from typing import Tuple
import pathlib

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [6]:
kx = np.arange(N).astype(np.float64) - N/2.0
ky, kx = np.meshgrid(kx, kx)
k = kx**2 + ky**2
# Define Kaiser-Squires kernel
D = np.zeros((N, N), dtype=np.complex128)
# D = np.where(k > 0, ((kx ** 2.0 - ky ** 2.0) + 1j * (2.0 * kx * ky))/k, D)
# Another formulation to avoid divide by zero warning
D[k>0] = (((kx ** 2.0 - ky ** 2.0) + 1j * (2.0 * kx * ky))[k>0]/k[k>0])
# Apply inverse FFT shift 
print(np.fft.ifftshift(D))

[[ 0.        +0.j         -1.        +0.j         -1.        +0.j
  ... -1.        +0.j         -1.        +0.j
  -1.        +0.j        ]
 [ 1.        +0.j          0.        +1.j         -0.6       +0.8j
  ... -0.8       -0.6j        -0.6       -0.8j
   0.        -1.j        ]
 [ 1.        +0.j          0.6       +0.8j         0.        +1.j
  ... -0.38461538-0.92307692j  0.        -1.j
   0.6       -0.8j       ]
 ...
 [ 1.        +0.j          0.8       -0.6j         0.38461538-0.92307692j
  ...  0.        +1.j          0.38461538+0.92307692j
   0.8       +0.6j       ]
 [ 1.        +0.j          0.6       -0.8j         0.        -1.j
  ... -0.38461538+0.92307692j  0.        +1.j
   0.6       +0.8j       ]
 [ 1.        +0.j          0.        -1.j         -0.6       -0.8j
  ... -0.8       +0.6j        -0.6       +0.8j
   0.        +1.j        ]]
